# Calculate the percentage of anomaly
#### Jose Valles (jose.valles.leon@gmail.com)

### Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import numpy as np
import calendar

sns.set()

from IPython.display import HTML

Import basin level code 2 and 3

In [2]:
BASIN_LEVEL3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',index_col="Codigo")
BASIN_LEVEL2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',index_col="Codigo")

In [3]:
def importmodelvariable(model_variable):
    df = pd.read_csv(f'../output_modelo/{model_variable}.csv')
    df = df.rename(columns={'-1': 'year','-1.1':'month'})
    df['date'] = pd.to_datetime(dict(year=df['year'],month=df['month'],day=1))
    df = df.set_index('date')
    df['days_in_month'] = df.index.days_in_month
    return df

def convertRunoff2Discharge(df_runoff):
    df_runoff_selected = df_runoff.drop(['year','month','days_in_month'],axis=1)
    df_discharge = pd.DataFrame(df_runoff_selected.values*1000*BASIN_LEVEL3.values,columns=df_runoff_selected.columns)
    df_discharge['days_in_month'] = df_runoff['days_in_month'].values
    df_discharge = df_discharge.loc[:, df_discharge.columns != 'days_in_month'].divide(df_discharge["days_in_month"]*24*3600, axis="index")
    df_discharge['date'] = df_runoff.index.values
    df_discharge = df_discharge.set_index('date')
    df_discharge['year'] = df_runoff['year'].values
    df_discharge['month'] = df_runoff['month'].values
    return df_discharge

In [35]:
hydrological_variable = ['Escorrentia_total','Escorrentia_sup','Escorrentia_sub','Pmedias','ETPmedias','ETR','HumedadSuelo']

for hydro in hydrological_variable:
    if hydro == "Escorrentia_total":
        RUNOFF_total = importmodelvariable(hydro)
    elif hydro == "Pmedias":
        PRECIP = importmodelvariable(hydro)
    elif hydro == "ETR":
        ETR = importmodelvariable(hydro)
    elif hydro == "HumedadSuelo":
        SM = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sup':
        RUNOFF_sup = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sub':
        RUNOFF_sub = importmodelvariable(hydro)
    elif hydro == 'ETPmedias':
        ETP = importmodelvariable(hydro)

In [6]:
HTML(RUNOFF_total.tail(6).to_html(index=False))

year,month,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,days_in_month
2023,1,1.44460,1.44550,1.01010,0.92577,0.76100,0.89067,1.46070,0.941440,0.82929,0.834950,0.855760,0.883610,1.01580,1.12000,1.10640,1.71930,2.349600,2.424400,2.29990,2.151400,1.84460,1.96890,2.333900,2.377000,3.3996,2.718600,2.858400,2.813400,2.65090,2.63790,2.37260,2.63010,2.68120,2.55630,2.63610,2.67250,2.64860,2.51890,2.55380,2.37880,2.16010,1.64530,1.63000,1.9348,0.69787,0.65244,1.4819,1.36910,0.64772,0.59043,1.02310,1.12130,0.528320,0.411650,0.50927,0.020014,0.000354,0.000674,5.666700e-05,4.173600e-05,7.807700e-06,1.060200e-05,7.714800e-06,0.000178,4.000400e-05,1.471100e-05,1.172500e-05,9.770600e-06,1.226200e-05,1.105400e-05,7.799000e-06,7.870200e-06,1.112400e-05,1.215500e-05,1.517200e-05,1.457900e-05,1.451100e-05,1.422500e-05,0.000344,0.000341,0.000307,0.000337,0.000218,0.000167,3.566300e-05,5.545000e-05,1.227500e-05,1.319100e-05,1.250900e-05,2.471100e-05,1.308800e-04,0.000171,9.877300e-05,3.082100e-05,5.561700e-05,5.433600e-05,3.744800e-05,0.032537,0.042254,0.038317,0.030925,0.071899,0.036890,0.231810,0.200000,0.223700,0.179240,0.133090,0.38975,0.304340,0.283420,0.298360,0.32351,0.250740,0.229230,0.264900,0.263990,0.194120,0.192970,0.242390,0.430890,0.51669,0.85219,0.930780,0.949930,0.86727,0.79212,1.001400,1.076700,1.19720,1.54540,1.638100,1.83760,1.848000,1.89870,2.15810,2.16780,1.93890,2.40430,2.55680,1.57300,1.00470,1.65150,1.59480,1.45690,1.36820,1.41110,1.41600,1.4215,1.39390,1.06470,1.15610,0.95176,1.01610,1.02600,1.15800,1.13620,0.75886,1.02060,0.65622,0.57915,0.50510,0.89827,0.63777,1.08290,0.93378,1.83490,1.0638,0.74024,0.59363,0.60306,0.67277,0.70841,0.90812,1.18570,1.24330,1.60030,1.40890,1.05460,1.36240,1.85940,2.45280,2.42050,2.47320,1.556200,1.523200,1.454400,1.446800,1.548500,1.60350,1.46190,1.29480,0.95413,1.618300,1.602800,1.542400,1.63710,1.76940,1.99820,2.353200,1.881700,2.199900,2.131900,2.313700,2.334400,2.298800,2.278300,2.095800,2.212400,1.976700,1.767200,1.505100,1.196700,1.923200,1.91490,1.133000,0.876270,0.977590,1.15420,0.99390,1.075800,1.05900,0.87406,0.910450,0.86223,1.152200,0.834990,0.748220,0.72656,0.81019,0.73608,0.55963,0.44189,0.178660,0.40267,0.020396,0.000347,0.115280,0.000331,0.035492,0.119230,0.41942,0.001067,0.103190,0.170900,0.78117,0.64430,0.046855,0.025411,0.30520,0.010666,0.004325,0.004249,0.002694,0.42799,0.44247,0.41569,0.273220,0.285550,0.41475,0.225950,0.209840,0.137780,0.102020,1.2956,0.45406,0.083342,0.000296,0.130590,0.065264,0.039159,1.040000e-04,0.006325,0.005518,0.001841,3.821500e-05,5.406200e-05,0.000409,0.000336,0.000373,0.000425,0.000406,0.000336,0.000257,8.236300e-05,1.048700e-04,2.083200e-05,2.425900e-05,1.325400e-05,2.678700e-05,0.007119,0.000908,0.020923,7.349300e-06,2.352400e-05,0.014240,31
2023,2,0.53677,0.53711,0.37532,0.34400,0.28277,0.33095,0.54278,0.349820,0.30815,0.31

In [9]:
PRECIP

,year,month,101,102,103,105,106,107,108,109,...,661,662,666,670,673,677,680,683,687,days_in_month
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,1980,1,21.677,16.754,8.1403,4.1573,2.9772,6.5288,8.1699,2.2236,...,19.3960,20.0240,19.2110,19.3250,19.2080,19.579,19.3850,19.4930,19.8980,31
1980-02-01,1980,2,74.517,90.112,131.6900,157.7100,166.9800,140.0500,125.9800,167.5100,...,155.8600,161.8700,153.5500,155.4200,153.7300,158.420,156.2600,156.7700,160.9100,29
1980-03-01,1980,3,176.440,172.180,152.6000,137.7700,132.0500,148.4400,157.3900,132.9800,...,85.8340,88.6790,84.9750,85.5230,84.9690,86.695,85.8060,86.2740,88.1170,31
1980-04-01,1980,4,220.510,178.850,112.8500,85.6020,78.2810,100.7800,110.4000,71.0090,...,254.6700,249.7900,256.2400,255.1700,256.2100,253.060,254.6400,253.9200,250.7100,30
1980-05-01,1980,5,84.370,87.146,95.5640,101.1500,103.1900,97.2850,94.1490,103.1500,...,142.9900,142.8900,142.9900,143.0100,143.0000,143.000,143.0100,142.9700,142.9200,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-01,2023,2,62.534,54.359,37.2720,20.0890,19.3910,31.4080,52.3180,61.6570,...,56.0710,54.9050,56.1730,54.6160,55.3460,52.019,54.6310,53.7530,51.5430,28
2023-03-01,2023,3,173.930,193.440,239.3300,240.0200,237.6400,229.8700,167.5800,141.7800,...,96.9400,94.2750,96.9550,92.7210,94.5900,85.463,92.8300,90.3960,84.2960,31
2023-04-01,2023,4,22.405,13.270,7.5846,9.6145,10.8690,9.0914,17.8140,50.5310,...,64.7350,68.2990,64.7800,70.6480,68.0890,80.753,70.4800,73.8630,82.3370,30


In [36]:
ETP

,year,month,101,102,103,105,106,107,108,109,...,661,662,666,670,673,677,680,683,687,days_in_month
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,1980,1,178.020,179.420,180.990,182.100,182.640,181.910,182.310,184.260,...,200.980,200.970,200.990,200.960,200.980,200.930,200.960,200.960,200.920,31
1980-02-01,1980,2,132.600,133.590,134.690,135.480,135.860,135.340,135.620,136.990,...,148.990,148.970,148.990,148.970,148.980,148.950,148.970,148.970,148.940,29
1980-03-01,1980,3,105.620,106.470,107.420,108.100,108.430,107.990,108.230,109.430,...,115.040,115.140,115.020,115.020,115.010,115.040,115.020,115.040,115.090,31
1980-04-01,1980,4,66.011,66.326,66.680,66.931,67.053,66.889,66.980,67.424,...,70.010,70.034,70.004,70.002,70.002,70.005,70.003,70.009,70.018,30
1980-05-01,1980,5,33.936,34.206,34.509,34.724,34.828,34.688,34.767,35.147,...,37.012,37.041,37.006,37.005,37.004,37.010,37.006,37.013,37.025,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-01,2023,2,128.760,131.120,134.980,137.040,137.980,136.600,137.010,141.500,...,145.130,145.400,145.070,145.090,145.060,145.170,145.100,145.160,145.320,28
2023-03-01,2023,3,113.730,115.160,117.480,118.800,119.460,118.640,119.030,122.160,...,122.180,122.550,122.100,122.120,122.080,122.240,122.140,122.230,122.440,31
2023-04-01,2023,4,63.774,64.729,66.270,67.148,67.591,67.050,67.318,69.323,...,55.123,55.373,55.066,55.089,55.060,55.173,55.106,55.160,55.312,30


### Convert from runoff to discharge

In [7]:
DISCHARGE = convertRunoff2Discharge(RUNOFF_total)

In [8]:
HTML(DISCHARGE.tail(6).to_html(index=True))

,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,year,month
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.453056,0.545085,0.306605,0.061179,0.118196,0.490494,1.319778,0.378910,0.018268,0.142463,0.025560,0.079836,0.080402,0.023417,0.123512,0.758099,0.371073,0.429049,0.742762,0.315674,0.152890,0.714520,0.419133,1.112000,0.704442,3.745383,0.798269,0.834020,0.728443,0.257053,0.008858,0.232726,1.069116,0.303503,1.679057,0.750343,0.951297,0.078998,0.137301,0.230917,0.700032,0.220528,1.020576,1.432463,0.090413,0.062360,0.896310,0.328678,0.067713,0.099860,0.302148,0.213928,0.011243,0.088988,0.467934,0.004805,1.718869e-06,0.000785,5.881655e-06,3.630708e-06,2.623555e-08,2.002917e-06,2.900539e-06,5.063829e-06,2.718312e-06,5.437534e-07,4.202509e-07,3.793841e-07,9.018869e-07,9.120871e-07,1.741264e-06,2.056877e-08,8.347984e-07,6.943380e-07,1.036617e-06,8.709080e-08,2.546360e-07,1.688900e-06,9.506392e-06,2.928498e-06,1.732576e-05,1.170312e-05,2.299989e-05,2.393405e-05,7.855873e-07,1.298057e-05,4.949597e-07,2.511727e-07,7.939553e-08,4.889796e-07,1.900848e-05,3.282971e-05,1.312843e-05,1.208261e-06,3.799997e-06,2.272115e-06,1.677778e-06,0.000668,0.001073,0.002360,0.002806,0.022710,0.000275,0.055737,0.004928,0.001754,0.000937,0.001640,0.027794,0.008636,0.017248,0.013590,0.045536,0.014510,0.002311,0.003165,0.006111,0.009567,0.002161,0.007059,0.005470,0.137352,0.004454,0.013553,0.002483,0.280412,0.144323,0.048978,0.010050,0.317805,0.320804,0.023852,0.832903,0.071066,0.974728,0.099912,0.497759,1.400027,0.090664,0.090687,1.099409,0.036386,0.154766,0.152430,0.175150,0.357579,0.478902,0.031192,0.160280,0.411654,0.094211,0.369482,0.007107,0.091807,0.060141,0.244276,0.100962,0.052982,0.070875,0.017150,0.046922,0.132762,0.278026,0.351221,0.578970,0.071470,0.160992,0.181113,0.168864,0.239145,0.360476,0.774901,0.318975,0.366516,0.054451,1.222690,0.155346,0.319296,0.091348,0.514257,0.642154,0.069599,0.005422,0.341653,0.199870,0.628980,0.369247,1.050637,0.562534,0.717217,0.711192,0.595577,0.939738,0.079755,0.071810,0.218253,0.353898,0.233198,0.320052,0.583380,2.504578,0.092812,0.373305,0.643558,0.658032,0.971566,0.733234,0.036777,0.310582,0.307014,1.857988,1.303701,0.130018,0.388460,1.364824,0.280035,0.294118,0.620483,0.462387,0.424515,0.253044,0.004745,0.256174,0.041131,0.083377,0.834123,0.215731,0.116211,0.054525,0.175444,0.133563,0.368782,0.171417,0.032018,0.236033,0.008734,0.000184,0.068392,0.000313,0.022222,0.022747,0.002505,0.001317,0.018454,0.004594,0.803219,0.262926,0.007225,0.002581,0.098680,0.000693,0.001321,0.002805,0.000426,0.042825,0.138107,0.063012,0.004284,0.019403

### Pleasee select the runoff type for the analysis (RUNOFF_total, RUNOFF_sub, RUNOFF_sup)

In [10]:
# Select the runoff type 
# RUNOFF = RUNOFF_total

### Select reference period from 'year_start' to 'year_end'

Select variable of interest

In [43]:
year_start = 1981
year_end = 2010
# Precip
SELECTED_REF_PRECIP = PRECIP[(PRECIP['year'] >= year_start) & (PRECIP['year'] <= year_end)]

# ETP
SELECTED_REF_ETP = ETP[(ETP['year'] >= year_start) & (ETP['year'] <= year_end)]

In [12]:
SELECTED_REF_PRECIP

,year,month,101,102,103,105,106,107,108,109,...,661,662,666,670,673,677,680,683,687,days_in_month
date,,,,,,,,,,,,,,,,,,,,,
1981-01-01,1981,1,152.620,159.220,160.390,194.350,189.880,173.570,195.320,173.260,...,151.4600,149.440,151.4000,147.9500,149.440,142.000,148.0600,146.070,141.080,31
1981-02-01,1981,2,221.550,229.280,198.520,214.390,216.110,213.690,263.600,252.250,...,125.3300,123.290,125.6600,123.3900,124.530,119.740,123.3700,122.120,118.960,28
1981-03-01,1981,3,72.583,76.544,67.444,70.558,77.714,77.053,83.214,75.141,...,70.8070,71.739,70.5130,71.1000,70.710,71.895,71.1620,71.449,72.199,31
1981-04-01,1981,4,81.443,81.681,55.207,66.241,83.043,79.841,122.200,131.780,...,78.0050,79.325,78.0460,80.2920,79.323,84.178,80.2220,81.521,84.773,30
1981-05-01,1981,5,214.420,251.690,197.470,105.310,115.030,179.280,230.700,183.320,...,295.6100,292.840,296.4400,294.5500,295.760,291.910,294.3800,293.440,290.990,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-08-01,2010,8,41.074,26.924,19.372,28.012,29.549,25.591,34.057,28.979,...,71.8490,76.543,71.0870,76.2670,73.646,84.604,76.3290,79.178,86.396,31
2010-09-01,2010,9,111.350,117.430,143.680,144.220,136.440,134.100,98.341,70.940,...,30.8310,48.124,29.1770,51.8260,41.121,89.500,51.6570,64.403,96.513,30
2010-10-01,2010,10,23.910,27.093,24.950,26.102,26.626,26.417,29.751,27.380,...,29.3370,31.051,29.2850,31.8760,30.713,36.284,31.8200,33.302,37.019,31


In [44]:
SELECTED_REF_ETP

,year,month,101,102,103,105,106,107,108,109,...,661,662,666,670,673,677,680,683,687,days_in_month
date,,,,,,,,,,,,,,,,,,,,,
1981-01-01,1981,1,146.250,147.320,148.520,149.370,149.780,149.220,149.520,150.990,...,167.940,167.840,167.960,167.940,167.960,167.880,167.920,167.910,167.820,31
1981-02-01,1981,2,119.460,120.510,121.690,122.530,122.940,122.390,122.690,124.170,...,133.030,133.110,133.010,133.010,133.010,133.010,133.010,133.030,133.050,28
1981-03-01,1981,3,94.466,95.064,95.738,96.219,96.457,96.145,96.324,97.187,...,97.076,97.235,97.040,97.049,97.033,97.095,97.058,97.094,97.183,31
1981-04-01,1981,4,59.692,59.658,59.621,59.596,59.586,59.604,59.598,59.564,...,56.036,56.107,56.020,56.028,56.019,56.055,56.034,56.049,56.095,30
1981-05-01,1981,5,45.324,45.219,45.104,45.027,44.995,45.048,45.029,44.914,...,36.088,36.259,36.048,36.069,36.046,36.134,36.082,36.119,36.231,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-08-01,2010,8,42.253,42.835,43.781,44.302,44.551,44.217,44.348,45.481,...,42.020,42.061,42.011,42.013,42.009,42.026,42.016,42.025,42.049,31
2010-09-01,2010,9,66.285,67.177,68.640,69.408,69.748,69.225,69.360,70.957,...,70.014,70.044,70.007,70.009,70.006,70.017,70.011,70.017,70.034,30
2010-10-01,2010,10,107.810,109.290,111.730,113.010,113.580,112.710,112.930,115.540,...,107.020,107.060,107.010,107.020,107.010,107.030,107.020,107.030,107.050,31


## Make a water balance for the whole country (Uruguay)

#### Annual and Monthly Analysis

In [130]:
# Obtener el promedio por mes
PRECIP_means_by_month = SELECTED_REF_PRECIP.iloc[:, 2:].groupby(SELECTED_REF_PRECIP.index.month).mean()

In [131]:
PRECIP_means_by_month

,101,102,103,105,106,107,108,109,110,111,...,661,662,666,670,673,677,680,683,687,days_in_month
date,,,,,,,,,,,,,,,,,,,,,
1,133.411700,138.592400,144.498800,149.127367,157.473633,153.337067,148.981967,163.741467,158.600067,165.273300,...,90.778667,91.197600,90.702653,91.140200,90.914200,91.835900,91.147767,91.387300,91.992900,31.000000
2,152.859200,152.013800,151.108200,142.673667,143.600633,149.151600,146.047500,137.259867,147.723033,136.617333,...,97.934167,98.131467,97.772000,97.584733,97.592900,97.144900,97.631800,97.496133,97.172667,28.233333
3,139.697933,142.953000,143.680767,148.185733,154.330433,151.743233,146.804000,137.905633,144.720233,135.536700,...,117.126850,116.846077,117.093240,116.534267,116.763877,115.552233,116.558200,116.231900,115.415667,31.000000
4,168.242467,170.727467,174.011867,178.763067,184.975233,181.363767,179.091600,186.780667,178.319500,186.714033,...,104.269157,105.373423,104.210070,105.797177,105.071607,108.478330,105.770673,106.674017,108.942160,30.000000
5,141.359483,131.714397,130.523100,121.499567,121.955000,127.007900,122.034557,117.596223,112.562753,116.519457,...,90.665467,90.855033,90.600933,90.705633,90.631100,90.839667,90.719967,90.769400,90.899400,31.000000
6,99.938467,95.450400,91.832667,87.401567,86.913267,89.859533,92.331567,90.422910,85.781817,90.020690,...,86.126903,87.037053,86.042597,87.243307,86.677533,89.244167,87.233870,87.910733,89.614743,30.000000
7,91.484167,79.927767,78.442633,74.204730,74.144063,76.108020,70.072723,64.129733,59.074217,62.322703,...,71.834233,73.278633,71.656500,73.428400,72.570433,76.341667,73.428133,74.417333,76.913633,31.000000
8,75.549413,66.520047,62.626173,62.292287,62.548397,62.805797,64.888810,65.056803,56.795673,64.367672,...,73.373800,74.962800,73.214933,75.275200,74.297133,78.696733,75.262067,76.420033,79.338667,31.000000
9,122.179600,113.900000,113.427500,106.827827,108.973180,112.266600,106.135333,103.803657,99.058680,102.813507,...,76.308930,77.884600,76.104483,78.000033,77.074667,81.104367,78.003067,79.058000,81.723867,30.000000


In [132]:
PRECIP_means_by_month = PRECIP_means_by_month.iloc[:, :-1]

In [133]:
PRECIP_means_by_month

,101,102,103,105,106,107,108,109,110,111,...,660,661,662,666,670,673,677,680,683,687
date,,,,,,,,,,,,,,,,,,,,,
1,133.411700,138.592400,144.498800,149.127367,157.473633,153.337067,148.981967,163.741467,158.600067,165.273300,...,91.276533,90.778667,91.197600,90.702653,91.140200,90.914200,91.835900,91.147767,91.387300,91.992900
2,152.859200,152.013800,151.108200,142.673667,143.600633,149.151600,146.047500,137.259867,147.723033,136.617333,...,98.504633,97.934167,98.131467,97.772000,97.584733,97.592900,97.144900,97.631800,97.496133,97.172667
3,139.697933,142.953000,143.680767,148.185733,154.330433,151.743233,146.804000,137.905633,144.720233,135.536700,...,117.017947,117.126850,116.846077,117.093240,116.534267,116.763877,115.552233,116.558200,116.231900,115.415667
4,168.242467,170.727467,174.011867,178.763067,184.975233,181.363767,179.091600,186.780667,178.319500,186.714033,...,105.211773,104.269157,105.373423,104.210070,105.797177,105.071607,108.478330,105.770673,106.674017,108.942160
5,141.359483,131.714397,130.523100,121.499567,121.955000,127.007900,122.034557,117.596223,112.562753,116.519457,...,90.971133,90.665467,90.855033,90.600933,90.705633,90.631100,90.839667,90.719967,90.769400,90.899400
6,99.938467,95.450400,91.832667,87.401567,86.913267,89.859533,92.331567,90.422910,85.781817,90.020690,...,86.994127,86.126903,87.037053,86.042597,87.243307,86.677533,89.244167,87.233870,87.910733,89.614743
7,91.484167,79.927767,78.442633,74.204730,74.144063,76.108020,70.072723,64.129733,59.074217,62.322703,...,73.328833,71.834233,73.278633,71.656500,73.428400,72.570433,76.341667,73.428133,74.417333,76.913633
8,75.549413,66.520047,62.626173,62.292287,62.548397,62.805797,64.888810,65.056803,56.795673,64.367672,...,74.922500,73.373800,74.962800,73.214933,75.275200,74.297133,78.696733,75.262067,76.420033,79.338667
9,122.179600,113.900000,113.427500,106.827827,108.973180,112.266600,106.135333,103.803657,99.058680,102.813507,...,77.969000,76.308930,77.884600,76.104483,78.000033,77.074667,81.104367,78.003067,79.058000,81.723867


In [134]:
# Seleccionar las últimas 6 filas
PRECIP_last_6 = PRECIP_means_by_month.tail(6)

In [135]:
PRECIP_last_6

,101,102,103,105,106,107,108,109,110,111,...,660,661,662,666,670,673,677,680,683,687
date,,,,,,,,,,,,,,,,,,,,,
7,91.484167,79.927767,78.442633,74.204730,74.144063,76.108020,70.072723,64.129733,59.074217,62.322703,...,73.328833,71.834233,73.278633,71.656500,73.428400,72.570433,76.341667,73.428133,74.417333,76.913633
8,75.549413,66.520047,62.626173,62.292287,62.548397,62.805797,64.888810,65.056803,56.795673,64.367672,...,74.922500,73.373800,74.962800,73.214933,75.275200,74.297133,78.696733,75.262067,76.420033,79.338667
9,122.179600,113.900000,113.427500,106.827827,108.973180,112.266600,106.135333,103.803657,99.058680,102.813507,...,77.969000,76.308930,77.884600,76.104483,78.000033,77.074667,81.104367,78.003067,79.058000,81.723867
10,135.545633,134.599133,132.703867,133.756833,136.095533,135.609233,136.165367,136.337067,140.919300,136.539133,...,104.764907,103.319320,104.700720,103.143050,104.818537,104.003003,107.567300,104.819750,105.754137,108.112033
11,139.398330,144.671117,147.004493,141.056100,145.627200,148.563087,145.757873,152.548613,147.933867,153.397853,...,99.825153,99.353756,99.933280,99.330590,100.192143,99.803087,101.653353,100.175353,100.665997,101.899890
12,123.199000,122.557200,120.935670,112.277760,115.070820,119.986677,119.642733,125.798900,132.385100,128.104467,...,78.012700,77.002980,77.806200,76.836290,77.612067,77.196133,78.822100,77.635067,78.052567,79.116667


In [136]:
ETP_means_by_month = SELECTED_REF_ETP.iloc[:, 2:].groupby(SELECTED_REF_PRECIP.index.month).mean()

In [137]:
ETP_means_by_month = ETP_means_by_month.iloc[:, :-1]

In [138]:
ETP_means_by_month

,101,102,103,105,106,107,108,109,110,111,...,660,661,662,666,670,673,677,680,683,687
date,,,,,,,,,,,,,,,,,,,,,
1,165.464333,167.870667,171.492667,173.526667,174.526000,173.265000,173.859667,178.189000,179.662667,179.975667,...,172.245667,172.205667,172.228000,172.201333,172.192333,172.196333,172.185667,172.190000,172.197667,172.196000
2,127.003500,128.790333,131.536000,133.066667,133.807667,132.841000,133.258667,136.495667,137.482667,137.731333,...,126.912367,126.799933,126.871933,126.783000,126.784467,126.779400,126.801467,126.788900,126.804933,126.841000
3,109.125667,110.350300,112.203567,113.249167,113.761667,113.109333,113.411533,115.617767,116.245600,116.427033,...,104.201667,104.047667,104.147700,104.025633,104.033133,104.021200,104.063600,104.039067,104.060600,104.118533
4,66.533733,66.874200,67.409933,67.713800,67.862500,67.670400,67.755900,68.414233,68.673533,68.712000,...,60.363200,60.190467,60.304067,60.164367,60.176333,60.161933,60.216467,60.184300,60.208933,60.280567
5,41.954133,42.070100,42.256067,42.349633,42.389700,42.325867,42.339567,42.525833,42.540067,42.560967,...,34.991467,34.823500,34.934500,34.797933,34.811200,34.796167,34.852633,34.819533,34.843267,34.915767
6,28.089433,28.136100,28.268067,28.316100,28.325800,28.279733,28.258667,28.360400,28.369633,28.378433,...,21.749133,21.588467,21.694967,21.563733,21.577700,21.562433,21.618600,21.585800,21.608433,21.679433
7,33.138233,33.460300,34.018000,34.307700,34.437967,34.242533,34.294467,34.913067,35.052633,35.111333,...,26.763233,26.616533,26.713700,26.594133,26.606133,26.592733,26.642767,26.613433,26.634167,26.697933
8,51.386133,52.060867,53.161133,53.771333,54.067567,53.679967,53.842167,55.173867,55.616233,55.720967,...,44.338533,44.159267,44.277033,44.132200,44.144567,44.129567,44.186033,44.152800,44.178400,44.252433
9,71.595367,72.797900,74.713000,75.759567,76.256933,75.584500,75.844167,78.071200,78.715133,78.896500,...,67.245333,67.085133,67.189900,67.061133,67.070600,67.058233,67.105367,67.077600,67.100533,67.164200


In [139]:
# Seleccionar las últimas 6 filas
ETP_last_6 = ETP_means_by_month.tail(6)

In [140]:
ETP_last_6

,101,102,103,105,106,107,108,109,110,111,...,660,661,662,666,670,673,677,680,683,687
date,,,,,,,,,,,,,,,,,,,,,
7,33.138233,33.460300,34.018000,34.307700,34.437967,34.242533,34.294467,34.913067,35.052633,35.111333,...,26.763233,26.616533,26.713700,26.594133,26.606133,26.592733,26.642767,26.613433,26.634167,26.697933
8,51.386133,52.060867,53.161133,53.771333,54.067567,53.679967,53.842167,55.173867,55.616233,55.720967,...,44.338533,44.159267,44.277033,44.132200,44.144567,44.129567,44.186033,44.152800,44.178400,44.252433
9,71.595367,72.797900,74.713000,75.759567,76.256933,75.584500,75.844167,78.071200,78.715133,78.896500,...,67.245333,67.085133,67.189900,67.061133,67.070600,67.058233,67.105367,67.077600,67.100533,67.164200
10,104.465267,106.091900,108.600533,109.993400,110.669667,109.797000,110.181133,113.147733,114.062567,114.301167,...,101.263933,101.112533,101.210800,101.091400,101.096867,101.087900,101.126367,101.103667,101.125000,101.180267
11,132.279100,133.981333,136.538333,137.959333,138.661000,137.788667,138.207000,141.254000,142.281000,142.521000,...,132.867333,132.763000,132.829333,132.749000,132.748667,132.743667,132.765000,132.753333,132.766667,132.800000
12,157.825000,160.041667,163.382000,165.265000,166.199667,165.042667,165.611000,169.692667,171.399000,171.638333,...,165.069000,164.935000,165.018333,164.917333,164.914667,164.909667,164.928333,164.916667,164.937000,164.971667


## Escenario 1

PRECIPITACION <br>
Jul: -50% normal <br>
Ago: normal <br>
Set: normal <br>
Oct: normal <br>
Nov: +25% normal <br>
Dic: +25% normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [206]:
# Crear una copia del DataFrame original
PRECIP_escenario_1 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.5
PRECIP_escenario_1.iloc[0] *= 0.5

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_1.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_1.iloc[4:6] *= 1.25

In [207]:
PRECIP_last_6

,101,102,103,105,106,107,108,109,110,111,...,660,661,662,666,670,673,677,680,683,687
date,,,,,,,,,,,,,,,,,,,,,
7,91.484167,79.927767,78.442633,74.204730,74.144063,76.108020,70.072723,64.129733,59.074217,62.322703,...,73.328833,71.834233,73.278633,71.656500,73.428400,72.570433,76.341667,73.428133,74.417333,76.913633
8,75.549413,66.520047,62.626173,62.292287,62.548397,62.805797,64.888810,65.056803,56.795673,64.367672,...,74.922500,73.373800,74.962800,73.214933,75.275200,74.297133,78.696733,75.262067,76.420033,79.338667
9,122.179600,113.900000,113.427500,106.827827,108.973180,112.266600,106.135333,103.803657,99.058680,102.813507,...,77.969000,76.308930,77.884600,76.104483,78.000033,77.074667,81.104367,78.003067,79.058000,81.723867
10,135.545633,134.599133,132.703867,133.756833,136.095533,135.609233,136.165367,136.337067,140.919300,136.539133,...,104.764907,103.319320,104.700720,103.143050,104.818537,104.003003,107.567300,104.819750,105.754137,108.112033
11,139.398330,144.671117,147.004493,141.056100,145.627200,148.563087,145.757873,152.548613,147.933867,153.397853,...,99.825153,99.353756,99.933280,99.330590,100.192143,99.803087,101.653353,100.175353,100.665997,101.899890
12,123.199000,122.557200,120.935670,112.277760,115.070820,119.986677,119.642733,125.798900,132.385100,128.104467,...,78.012700,77.002980,77.806200,76.836290,77.612067,77.196133,78.822100,77.635067,78.052567,79.116667


In [208]:
PRECIP_escenario_1

,101,102,103,105,106,107,108,109,110,111,...,660,661,662,666,670,673,677,680,683,687
date,,,,,,,,,,,,,,,,,,,,,
7,45.742083,39.963883,39.221317,37.102365,37.072032,38.054010,35.036362,32.064867,29.537108,31.161352,...,36.664417,35.917117,36.639317,35.828250,36.714200,36.285217,38.170833,36.714067,37.208667,38.456817
8,75.549413,66.520047,62.626173,62.292287,62.548397,62.805797,64.888810,65.056803,56.795673,64.367672,...,74.922500,73.373800,74.962800,73.214933,75.275200,74.297133,78.696733,75.262067,76.420033,79.338667
9,122.179600,113.900000,113.427500,106.827827,108.973180,112.266600,106.135333,103.803657,99.058680,102.813507,...,77.969000,76.308930,77.884600,76.104483,78.000033,77.074667,81.104367,78.003067,79.058000,81.723867
10,135.545633,134.599133,132.703867,133.756833,136.095533,135.609233,136.165367,136.337067,140.919300,136.539133,...,104.764907,103.319320,104.700720,103.143050,104.818537,104.003003,107.567300,104.819750,105.754137,108.112033
11,174.247912,180.838896,183.755617,176.320125,182.034000,185.703858,182.197342,190.685767,184.917333,191.747317,...,124.781442,124.192195,124.916600,124.163238,125.240179,124.753858,127.066692,125.219192,125.832496,127.374863
12,153.998750,153.196500,151.169588,140.347200,143.838525,149.983346,149.553417,157.248625,165.481375,160.130583,...,97.515875,96.253725,97.257750,96.045363,97.015083,96.495167,98.527625,97.043833,97.565708,98.895833


In [209]:
# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_1.reset_index(inplace=True)
PRECIP_escenario_1['date'] = pd.to_numeric(PRECIP_escenario_1['date'], errors='coerce')
PRECIP_escenario_1.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_1.insert(0, 'year', 2023)

In [210]:
PRECIP_E_1 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_1])
PRECIP_E_1.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

In [211]:
PRECIP_E_1.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_1.csv', index=False, float_format='%.4f')

In [212]:
# Crear una copia del DataFrame original
ETP_escenario_1 = ETP_last_6.copy()

In [213]:
# Multiplicar la primera fila por 0.5
ETP_escenario_1.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_1.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_1.iloc[4:6] *= 1

In [214]:
# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_1.reset_index(inplace=True)
ETP_escenario_1['date'] = pd.to_numeric(ETP_escenario_1['date'], errors='coerce')
ETP_escenario_1.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_1.insert(0, 'year', 2023)

In [215]:
ETP_E_1 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_1])
ETP_E_1.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

In [216]:
ETP_E_1.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_1.csv', index=False, float_format='%.4f')

## Escenario 2

PRECIPITACION <br>
Jul: -75% normal <br>
Ago: normal <br>
Set: normal <br>
Oct: normal <br>
Nov: +25% normal <br>
Dic: +25% normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [217]:
# Crear una copia del DataFrame original
PRECIP_escenario_2 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_2.iloc[0] *= 0.25

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_2.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_2.iloc[4:6] *= 1.25

In [218]:
# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_2.reset_index(inplace=True)
PRECIP_escenario_2['date'] = pd.to_numeric(PRECIP_escenario_2['date'], errors='coerce')
PRECIP_escenario_2.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_2.insert(0, 'year', 2023)

In [219]:
PRECIP_E_2 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_2])
PRECIP_E_2.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

In [220]:
PRECIP_E_2.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_2.csv', index=False, float_format='%.4f')

In [221]:
# Crear una copia del DataFrame original
ETP_escenario_2 = ETP_last_6.copy()

In [222]:
# Multiplicar la primera fila por 0.5
ETP_escenario_2.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_2.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_2.iloc[4:6] *= 1

In [223]:
# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_2.reset_index(inplace=True)
ETP_escenario_2['date'] = pd.to_numeric(ETP_escenario_2['date'], errors='coerce')
ETP_escenario_2.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_2.insert(0, 'year', 2023)

In [224]:
ETP_E_2 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_2])
ETP_E_2.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

In [225]:
ETP_E_2.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_2.csv', index=False, float_format='%.4f')

## Escenario 3

PRECIPITACION <br>
Jul: normal <br>
Ago: normal <br>
Set: normal <br>
Oct: normal <br>
Nov: +25% normal <br>
Dic: +25% normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [226]:
# Crear una copia del DataFrame original
PRECIP_escenario_3 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_3.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_3.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_3.iloc[4:6] *= 1.25

In [227]:
# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_3.reset_index(inplace=True)
PRECIP_escenario_3['date'] = pd.to_numeric(PRECIP_escenario_3['date'], errors='coerce')
PRECIP_escenario_3.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_3.insert(0, 'year', 2023)

PRECIP_E_3 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_3])
PRECIP_E_3.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_3.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_3.csv', index=False, float_format='%.4f')

In [228]:
# Crear una copia del DataFrame original
ETP_escenario_3 = ETP_last_6.copy()

In [229]:
# Multiplicar la primera fila por 0.5
ETP_escenario_3.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_3.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_3.iloc[4:6] *= 1

In [230]:
# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_3.reset_index(inplace=True)
ETP_escenario_3['date'] = pd.to_numeric(ETP_escenario_3['date'], errors='coerce')
ETP_escenario_3.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_3.insert(0, 'year', 2023)

In [231]:
ETP_E_3 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_3])
ETP_E_3.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

In [232]:
ETP_E_3.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_3.csv', index=False, float_format='%.4f')

## Escenario 4

PRECIPITACION <br>
Jul: normal <br>
Ago: normal <br>
Set: normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [233]:
# Crear una copia del DataFrame original
PRECIP_escenario_4 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_4.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_4.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_4.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_4.reset_index(inplace=True)
PRECIP_escenario_4['date'] = pd.to_numeric(PRECIP_escenario_4['date'], errors='coerce')
PRECIP_escenario_4.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_4.insert(0, 'year', 2023)

PRECIP_E_4 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_4])
PRECIP_E_4.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_4.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_4.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_4 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_4.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_4.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_4.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_4.reset_index(inplace=True)
ETP_escenario_4['date'] = pd.to_numeric(ETP_escenario_4['date'], errors='coerce')
ETP_escenario_4.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_4.insert(0, 'year', 2023)

ETP_E_4 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_4])
ETP_E_4.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_4.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_4.csv', index=False, float_format='%.4f')

## Escenario 5 

PRECIPITACION <br>
Jul: -50% normal <br>
Ago: normal <br>
Set: normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [234]:
# Crear una copia del DataFrame original
PRECIP_escenario_5 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_5.iloc[0] *= 0.5

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_5.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_5.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_5.reset_index(inplace=True)
PRECIP_escenario_5['date'] = pd.to_numeric(PRECIP_escenario_5['date'], errors='coerce')
PRECIP_escenario_5.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_5.insert(0, 'year', 2023)

PRECIP_E_5 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_5])
PRECIP_E_5.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_5.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_5.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_5 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_5.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_5.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_5.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_5.reset_index(inplace=True)
ETP_escenario_5['date'] = pd.to_numeric(ETP_escenario_5['date'], errors='coerce')
ETP_escenario_5.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_5.insert(0, 'year', 2023)

ETP_E_5 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_5])
ETP_E_5.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_5.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_5.csv', index=False, float_format='%.4f')

## Escenario 6

PRECIPITACION <br>
Jul: -75% normal <br>
Ago: normal <br>
Set: normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [235]:
# Crear una copia del DataFrame original
PRECIP_escenario_6 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_6.iloc[0] *= 0.25

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_6.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_6.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_6.reset_index(inplace=True)
PRECIP_escenario_6['date'] = pd.to_numeric(PRECIP_escenario_6['date'], errors='coerce')
PRECIP_escenario_6.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_6.insert(0, 'year', 2023)

PRECIP_E_6 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_6])
PRECIP_E_6.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_6.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_6.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_6 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_6.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_6.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_6.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_6.reset_index(inplace=True)
ETP_escenario_6['date'] = pd.to_numeric(ETP_escenario_6['date'], errors='coerce')
ETP_escenario_6.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_6.insert(0, 'year', 2023)

ETP_E_6 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_6])
ETP_E_6.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_6.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_6.csv', index=False, float_format='%.4f')

## Escenario 7

PRECIPITACION <br>
Jul: -50% normal <br>
Ago: -75% normal <br>
Set: normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [236]:
# Crear una copia del DataFrame original
PRECIP_escenario_7 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_7.iloc[0] *= 0.50

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_7.iloc[1] *= 0.25

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_7.iloc[2:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_7.reset_index(inplace=True)
PRECIP_escenario_7['date'] = pd.to_numeric(PRECIP_escenario_7['date'], errors='coerce')
PRECIP_escenario_7.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_7.insert(0, 'year', 2023)

PRECIP_E_7 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_7])
PRECIP_E_7.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_7.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_7.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_7 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_7.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_7.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_7.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_7.reset_index(inplace=True)
ETP_escenario_7['date'] = pd.to_numeric(ETP_escenario_7['date'], errors='coerce')
ETP_escenario_7.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_7.insert(0, 'year', 2023)

ETP_E_7 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_7])
ETP_E_7.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_7.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_7.csv', index=False, float_format='%.4f')

## Escenario 8

PRECIPITACION <br>
Jul: -75% normal <br>
Ago: -75% normal <br>
Set: normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [237]:
# Crear una copia del DataFrame original
PRECIP_escenario_8 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_8.iloc[0] *= 0.25

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_8.iloc[1] *= 0.25

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_8.iloc[2:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_8.reset_index(inplace=True)
PRECIP_escenario_8['date'] = pd.to_numeric(PRECIP_escenario_8['date'], errors='coerce')
PRECIP_escenario_8.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_8.insert(0, 'year', 2023)

PRECIP_E_8 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_8])
PRECIP_E_8.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_8.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_8.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_8 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_8.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_8.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_8.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_8.reset_index(inplace=True)
ETP_escenario_8['date'] = pd.to_numeric(ETP_escenario_8['date'], errors='coerce')
ETP_escenario_8.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_8.insert(0, 'year', 2023)

ETP_E_8 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_8])
ETP_E_8.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_8.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_8.csv', index=False, float_format='%.4f')

## Escenario 9

PRECIPITACION <br>
Jul: -50% normal <br>
Ago: -50% normal <br>
Set: normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [238]:
# Crear una copia del DataFrame original
PRECIP_escenario_9 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_9.iloc[0] *= 0.50

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_9.iloc[1] *= 0.50

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_9.iloc[2:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_9.reset_index(inplace=True)
PRECIP_escenario_9['date'] = pd.to_numeric(PRECIP_escenario_9['date'], errors='coerce')
PRECIP_escenario_9.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_9.insert(0, 'year', 2023)

PRECIP_E_9 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_9])
PRECIP_E_9.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_9.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_9.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_9 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_9.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_9.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_9.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_9.reset_index(inplace=True)
ETP_escenario_9['date'] = pd.to_numeric(ETP_escenario_9['date'], errors='coerce')
ETP_escenario_9.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_9.insert(0, 'year', 2023)

ETP_E_9 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_9])
ETP_E_9.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_9.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_9.csv', index=False, float_format='%.4f')

## Escenario 10

PRECIPITACION <br>
Jul: -75% normal <br>
Ago: normal <br>
Set: +25% normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [239]:
# Crear una copia del DataFrame original
PRECIP_escenario_10 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_10.iloc[0] *= 0.25

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_10.iloc[1] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_10.iloc[2] *= 1.25

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_10.iloc[3:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_10.reset_index(inplace=True)
PRECIP_escenario_10['date'] = pd.to_numeric(PRECIP_escenario_10['date'], errors='coerce')
PRECIP_escenario_10.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_10.insert(0, 'year', 2023)

PRECIP_E_10 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_10])
PRECIP_E_10.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_10.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_10.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_10 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_10.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_10.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_10.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_10.reset_index(inplace=True)
ETP_escenario_10['date'] = pd.to_numeric(ETP_escenario_10['date'], errors='coerce')
ETP_escenario_10.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_10.insert(0, 'year', 2023)

ETP_E_10 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_10])
ETP_E_10.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_10.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_10.csv', index=False, float_format='%.4f')

## Escenario 11

PRECIPITACION <br>
Jul: -50% normal <br>
Ago: normal <br>
Set: +25% normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [240]:
# Crear una copia del DataFrame original
PRECIP_escenario_11 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_11.iloc[0] *= 0.5

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_11.iloc[1] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_11.iloc[2] *= 1.25

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_11.iloc[3:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_11.reset_index(inplace=True)
PRECIP_escenario_11['date'] = pd.to_numeric(PRECIP_escenario_11['date'], errors='coerce')
PRECIP_escenario_11.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_11.insert(0, 'year', 2023)

PRECIP_E_11 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_11])
PRECIP_E_11.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_11.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_11.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_11 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_11.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_11.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_11.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_11.reset_index(inplace=True)
ETP_escenario_11['date'] = pd.to_numeric(ETP_escenario_11['date'], errors='coerce')
ETP_escenario_11.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_11.insert(0, 'year', 2023)

ETP_E_11 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_11])
ETP_E_11.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_11.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_11.csv', index=False, float_format='%.4f')

## Escenario 12 

PRECIPITACION <br>
Jul: -50% normal <br>
Ago: -50% normal <br>
Set: +25% normal <br>
Oct: normal <br>
Nov: normal <br>
Dic: normal <br>

EVAPOTRANSPIRACION <br>
Valor normal del mes correspondiente

In [241]:
# Crear una copia del DataFrame original
PRECIP_escenario_12 = PRECIP_last_6.copy()

# Multiplicar la primera fila por 0.25
PRECIP_escenario_12.iloc[0] *= 0.50

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_12.iloc[1] *= 0.50

# Multiplicar la segunda, tercera y cuarta fila por 1
PRECIP_escenario_12.iloc[2] *= 1.25

# Multiplicar la quinta y sexta fila por 1.25
PRECIP_escenario_12.iloc[3:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
PRECIP_escenario_12.reset_index(inplace=True)
PRECIP_escenario_12['date'] = pd.to_numeric(PRECIP_escenario_12['date'], errors='coerce')
PRECIP_escenario_12.rename(columns={'date': 'month'}, inplace=True)
PRECIP_escenario_12.insert(0, 'year', 2023)

PRECIP_E_12 = pd.concat([PRECIP.iloc[:, :-1], PRECIP_escenario_12])
PRECIP_E_12.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

PRECIP_E_12.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/Pmedias_E_12.csv', index=False, float_format='%.4f')

# Crear una copia del DataFrame original
ETP_escenario_12 = ETP_last_6.copy()

# Multiplicar la primera fila por 0.5
ETP_escenario_12.iloc[0] *= 1

# Multiplicar la segunda, tercera y cuarta fila por 1
ETP_escenario_12.iloc[1:4] *= 1

# Multiplicar la quinta y sexta fila por 1.25
ETP_escenario_12.iloc[4:6] *= 1

# Agregar una nueva columna con el valor '2023' como índice
ETP_escenario_12.reset_index(inplace=True)
ETP_escenario_12['date'] = pd.to_numeric(ETP_escenario_12['date'], errors='coerce')
ETP_escenario_12.rename(columns={'date': 'month'}, inplace=True)
ETP_escenario_12.insert(0, 'year', 2023)

ETP_E_12 = pd.concat([ETP.iloc[:, :-1], ETP_escenario_12])
ETP_E_12.rename(columns={'year': '-1', 'month': '-1.1'}, inplace=True)

ETP_E_12.to_csv('C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/escenarios/ETPmedias_E_12.csv', index=False, float_format='%.4f')

In [242]:
# # Supongamos que tienes un DataFrame llamado df

# # Seleccionar las últimas 5 filas
# last_5_rows = df.tail(5)

# # Seleccionar la primera fila
# first_row = df.head(1)

# # Combinar las filas en un nuevo DataFrame
# df_combined = pd.concat([last_5_rows, first_row])

In [243]:
# display(sum_avg_runoff_year.tail(5))

In [12]:
# sum_avg_runoff_month.to_clipboard()